# Validation: comparison with _ChromHMM_ emissions

### Imports and paths

In [ ]:
## imports

import numpy as np
import pickle
import os
from utils import *
from scipy.stats import pearsonr
rho = pearsonr

In [ ]:
## paths

root_path = './../'

# path to data, random splits, cell list
data_base_path = root_path+'data/'
split_base_path = data_base_path+'- splits/'
cell_list_path = data_base_path+'cells.csv'
name_base_path = data_base_path+'names.csv'
emission_path = data_base_path+'chromhmm/'+'emissions.tab'
color_path = data_base_path+'chromhmm/'+'states_colors.tab'

# path to folder where to write/read results
score_base_path = root_path+'scores/'
bests_path = score_base_path+'bests_{}.pkl'

# suffixes
data_suffix = ''
split_suffix = 'iteration_'
X_tail = 'X.csv'
T_tail = 'T.csv'

### Load and inspect _ChromHMM_ emission patterns

In [ ]:
# extract emissions
emissions, hm_index, hs_index = extract_emissions(emission_path)

# sort them according to our ordering
names = np.loadtxt(name_base_path, delimiter=',', dtype=str)
sorted_revindex = {names[n]: n for n in range(len(names))}
sorter = np.zeros(len(names), dtype=int)
for n in range(len(hm_index)):
    hm = hm_index[n]
    new_n = sorted_revindex[hm]
    sorter[new_n] = n
sorted_emissions = emissions.T[sorter]

# extract colors and state descriptions
index, description_dict, color_dict = extract_states_colors(color_path)

Let's plot here _ChromHMM_ emission patterns together.

In [ ]:
# plot them
states = np.asarray([description_dict[name] for name in [index[s] for s in range(1, len(index)+1)]], dtype=str)
plt.figure(dpi=100, figsize=(6, 2))
plt.imshow(sorted_emissions)
plt.title('emission patterns')
plt.yticks(range(len(names)), names, rotation=0)
plt.xticks(range(len(hs_index)), states, rotation=90)
plt.show()
plt.close()

Let's take a look at each pattern separately

In [ ]:
# plot each emission pattern separately for deeper inspection
min_val = sorted_emissions.min()
max_val = sorted_emissions.max()
for p, pattern in enumerate(sorted_emissions.T):
    name = index[p+1]
    color = color_dict[name]
    if p == sorted_emissions.shape[1] - 1:
        color = [channel - 0.1 for channel in color]
    x = np.arange(1, sorted_emissions.shape[0]+1)
    plt.figure(dpi=100, figsize=(6,2))
    plt.bar(x, pattern, color=color)
    plt.ylim([min_val - 0.1, max_val + 0.1])
    plt.xticks(x, names, rotation=90)
    plt.title(description_dict[name])
    plt.show()
    plt.close()

In the following, we are going to match _ShallowChrome_ patterns with the _ChromHMM_ emissions maximizing Pearson's correlation.

However, in all the cases where no Histone Mark activity is measured, _ShallowChrome_ patterns are null (zero vectors), and no correlation coefficient would be properly defined. Here we make the following choice: by default, we match all null _ShallowChrome_ patterns with the _ChromHMM_ emission with lowest intra-pattern norm and variance.

Let's find such a candidate!

In [ ]:
# let's inspect intra-pattern norm and std

intrapattern_devs = sorted_emissions.std(axis=0, keepdims=False)
dev_sorter = np.argsort(intrapattern_devs)

x = np.arange(1, sorted_emissions.shape[1]+1)
states = np.asarray([description_dict[name] for name in [index[s] for s in range(1, len(index)+1)]], dtype=str)

plt.figure(dpi=150, figsize=(6,2))
plt.plot(x, intrapattern_devs[dev_sorter], '|-', linewidth=0.9, color='cornflowerblue')
plt.plot([1], intrapattern_devs[dev_sorter][0]+0.05, 'v', color='indianred', markersize=8)
plt.xticks(x, states[dev_sorter], rotation=90)
plt.title('Intra-pattern standard-deviation')
plt.show()
plt.close()

intrapattern_norms = np.linalg.norm(sorted_emissions, ord=1, axis=0)
norm_sorter = np.argsort(intrapattern_norms)

x = np.arange(1, sorted_emissions.shape[1]+1)
states = np.asarray([description_dict[name] for name in [index[s] for s in range(1, len(index)+1)]], dtype=str)

plt.figure(dpi=150, figsize=(6,2))
plt.plot(x, intrapattern_norms[norm_sorter], '|-', linewidth=0.9, color='cornflowerblue')
plt.plot([1], intrapattern_norms[norm_sorter][0]+0.25, 'v', color='indianred', markersize=8)
plt.xticks(x, states[norm_sorter], rotation=90)
plt.title('Intra-pattern norms')
plt.show()
plt.close()

# save ChromHMM pattern with min emission norm; null ShallowChrome patterns will be matched with it
emissions_l1_norms = np.asarray([np.linalg.norm(em, 1) for em in sorted_emissions.T])
min_emission = np.argmin(emissions_l1_norms)

State `Quiescent/Low` has the lowest intra-pattern norm and variance, it will always match null _ShallowChrome_ patterns.

### Computation of _ShallowChrome_ patterns over the test set

Here we will compute _ShallowChrome_ patterns for the test set of the standard reference split `0`.
For each epigenome/cell, we will load the best models already trained in notebook `model inspection`.
Also, we will keep track of model logits for later validation.

In [ ]:
# load folders for data in best format
ref_iter = 0  # <- std split `0`
with open(bests_path.format(ref_iter), 'rb') as handle:
    best_formats = pickle.load(handle)
folders, cells = load_cell_paths(cell_list_path, data_base_path, suffix='/'+data_suffix, best_formats=best_formats)

# load std split `0`
split = load_split(split_base_path, split_suffix, ref_iter)

# enforce standard (hyper)parameters
penalty='l2'
Cs=[+np.inf]
random_state = 666

In [ ]:
# compute patterns on test set of split 0 for each cell line

patterns_by_cell = dict()
logits_by_cell = dict()
for c, cell in enumerate(cells):
    
    print '\r\tcell {0}...'.format(cell),
    
    # load data
    folder = folders[c]
    X = np.loadtxt(folder+X_tail, delimiter=',')
    X_test = X[split[-1]]
    
    # load model
    C = Cs[0]
    best = best_formats[cell]
    score_folder = score_base_path+str(cell)+'/'+str(best)+'/'
    with open(score_folder+'model_C_'+str(C)+'_iter_'+str(ref_iter)+'.pkl', 'rb') as model_file:
        model = pickle.load(model_file)

    # extract patterns
    _, b, psis = extract_patterns(model, X_test, collapse=False)
    patterns_by_cell[cell] = np.abs(psis)
    logits_by_cell[cell] = b + np.sum(psis, axis=1)
    
print ' done.'

### Find best matching ChromHMM states

Now we will compare the extracted _ShallowChrome_ patterns with _ChromHMM_ emissions to find, for each gene the best matching state/group of states.

In [ ]:
bmus_by_cell = dict()
for cell in cells:
    
    print '\r\tcell {0}...'.format(cell),
    
    patterns = patterns_by_cell[cell]
    bmus = np.ndarray((len(patterns),))
    correlations = np.ndarray((len(patterns), len(sorted_emissions.T)))
    for p, pattern in enumerate(patterns):
        
        pattern = pattern.copy()
        
        if pattern.sum() == 0.0:
            bmus[p] = min_emission
            correlation = np.zeros((len(sorted_emissions.T),))
            correlation[min_emission] = 1.0
            correlations[p] = correlation
            
        else:
            correlation = np.ndarray((len(sorted_emissions.T),))
            for e, em in enumerate(sorted_emissions.T):
                correlation[e] = rho(pattern, em)[0]
            bmus[p] = int(np.argmax(correlation))
            correlations[p] = correlation
        
    bmus_by_cell[cell] = bmus
    
print ' done.'

Coarse-grain matchings for group-level analysis.

In [ ]:
# prepare grouping for coarse-grained analysis

coarse_index = {
    1: 'Active',
    2: 'Bivalent',
    3: 'Enhancers',
    4: 'Repressed'}

grouping = {
    # Active
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    # Bivalent
    10: 2,
    11: 2,
    12: 2,
    # Enhancers
    6: 3,
    7: 3,
    # Repressed
    8: 4,
    9: 4,
    14: 4,
    13: 4,
    15: 4}

# group best matching units according to membership map above
def coarse_grain(bmus):
    coarser = [grouping[1+int(bmu)]-1 for bmu in bmus]
    return np.asarray(coarser)
groups_by_cell = {cell: coarse_grain(bmus_by_cell[cell]) for cell in cells}

### Compute group-level logit-based rankings of matched emissions

Let's now rank _ChromHMM_ state groups based on the mean _ShallowChrome_ logits of matched patterns.

In [ ]:
# compute mean logit values for each group and for each cell
mean_logits_by_cell = dict()
for target in cells:
    bmus = groups_by_cell[target]
    logits = logits_by_cell[target]
    mean_logits, _ = compute_mean_std_logits_per_state(logits, bmus, len(coarse_index))
    mean_logits_by_cell[target] = mean_logits
    
# given the above, compute rankings based on logit means per group
rank_by_cell = dict()
for target in cells:
    rank = np.zeros(len(coarse_index))
    scores = mean_logits_by_cell[target]
    arg = np.argsort(-scores)
    for i, a in enumerate(arg):
        if scores[a] == -np.inf:
            rank[a] = None
        else:
            rank[a] = i+1
    rank_by_cell[target] = rank
ranks = np.vstack([rank_by_cell[target][np.newaxis, :] for target in cells])

In [ ]:
# plot ranking histogram for each group
fig = plt.figure(dpi=300)
res = plt.hist(ranks,
               bins=4,
               color=['cadetblue', color_dict['Het'], color_dict['BivFlnk'], 'indianred'],
               label=['1st', '2nd', '3rd', '4th'])
plt.xticks(np.linspace(1.3, 3.6, 4), ['Active', 'Bivalent', 'Enhancers', 'Repressed'], rotation=0)
ticks = np.asarray([0, 14, 28, 42, 56])
plt.yticks(ticks, ['{:.0f}%'.format(t) for t in (100.0/56.0)*ticks])
plt.ylabel('Rank frequency')
plt.xlabel('Group')
plt.legend(loc=[0.7, 0.68])
fig.tight_layout()
plt.savefig('./ChromHMM_rankings.pdf', format='pdf')
plt.show()
plt.close()

Let's now perform finer-level analysis: ranks are computed at the level of single _ChromHMM_ states.

In [ ]:
# compute mean and std of logit values for each state and for each cell
mean_logits_by_cell = dict()
for target in cells:
    bmus = bmus_by_cell[target]
    logits = logits_by_cell[target]
    mean_logits, _ = compute_mean_std_logits_per_state(logits, bmus, len(index))
    mean_logits_by_cell[target] = mean_logits

# given the above, compute rankings based on logit means per state
rank_by_cell = dict()
for target in cells:
    rank = np.zeros(len(index))
    scores = mean_logits_by_cell[target]
    arg = np.argsort(-scores)
    for i, a in enumerate(arg):
        if scores[a] == -np.inf:
            rank[a] = None
        else:
            rank[a] = i+1
    rank_by_cell[target] = rank
ranks = np.vstack([rank_by_cell[target][np.newaxis, :] for target in cells])

# sort chromatin states based on their median ranking
median_ranks = np.zeros(len(index))
all_ranks = list()
for state in range(ranks.shape[1]):
    a = ranks[:,state]
    valid = np.where(~np.isnan(a))
    all_ranks.append(a[valid])
    median_ranks[state] = np.median(a[valid])
arg = np.argsort(median_ranks)

In [ ]:
# plot fine-graned rank analysis

fig = plt.figure(dpi=300, figsize=(7.5,5))
box = plt.boxplot(16 - np.asarray([all_ranks[i] for i in arg]), showfliers=False)
plt.xticks(range(1, 1+len(index)), [description_dict[index[a+1]] for a in arg], rotation=90)
plt.yticks(range(1, 1+len(index)), ['{}'.format(16-i) for i in range(1, 1+len(index))])
for item, line_list in box.items():
    for l, line in enumerate(line_list):
        if item=='medians':
            line.set_color(colors.to_hex(color_dict[index[arg[l]+1]]))
            if index[arg[l]+1] == 'Quies':
                line.set_color(colors.to_hex([0.9, 0.9, 0.9]))
            line.set_linewidth(1.5)
        else:
            line.set_color('grey')
            line.set_linewidth(0.7)
for r in range(1, 1+len(index)):
    plt.axhline(y=r, linestyle='--', linewidth=0.3, color='lightgrey')
plt.ylabel('Rank')
fig.tight_layout()
plt.savefig('./ChromHMM_rankings_fine.pdf', format='pdf')
plt.show()
plt.close()